## Importing Necessary Libraries

In [1]:
import selenium
import requests
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np


## Starting Browser Instance

In [2]:
#disabling notifications and starting maximised for better presentability
t1 =time.time()
options = Options()
# options.add_argument("--headless=new")
options.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 1 
})

#Starting browser instance
driver = webdriver.Chrome(options = options, service=Service(ChromeDriverManager().install()))
driver.get('https://www.amazon.in/s?rh=n%3A6612025031&fs=true&ref=lp_6612025031_sar')
time.sleep(5)#waiting for page to load
driver.maximize_window()
#including out of stock
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="p_n_availability/1318485031"]/span/a/span'))).click()




## Performing The Task

In [3]:
#Creating DataFrame
productdf = pd.DataFrame(columns = ['Name', 'Price', 'Rating', 'Seller', 'Ratings_cnt'])

In [4]:
product_index = 1001
iterate = True
while iterate:
    time.sleep(3)
    products = driver.find_elements(By.XPATH, "//a[@class = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for product in products:
        #opening product pages
        product.click()
    time.sleep(10)
    while len(driver.window_handles) > 1:
        #switiching tab to the product page
        current_tab = driver.window_handles[1]
        driver.switch_to.window(current_tab)
        #extracting title
        try:
            Title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'productTitle']")))
            Title = Title.text
        except:
            pass
            
        #extracting Rating
        Rating = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'acrPopover']/span/a/span")))
        Rating = float(Rating.text)
        
        #checking for unavailability
#         try:
#             abc = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="outOfStock"]/div/div[1]/span[1]'))).text
#             if abc == 'Currently unavailable.':
#                 Price = np.NaN
#                 Seller = np.NaN

#         except:
#             pass
        
        #extracting price
        try:
            price_decimal = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id = "corePrice_feature_div"]/div/div/span[1]/span[2]/span[2]')))
            Price = int(price_decimal.text.replace(',',''))
        except:
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'buybox-see-all-buying-choices']/span/a"))).click()
                time.sleep(2)
                Price = (WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[@id="aod-price-1"]/div[1]/span[2]/span[2]/span[2]'))).text.replace(',',''))
                Price = int(Price)
                driver.back()
            except:
                pass


        #extracting Seller
        try:
            Seller = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sellerProfileTriggerId"]'))).text
            Ratings_cnt = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="acrCustomerReviewText"]'))).text
            fastest_delivery = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE"]/span/span'))).text

        except:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'buybox-see-all-buying-choices']/span/a"))).click()
            Seller = np.Nan 
#             WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@id = 'aod-offer-soldBy']/div/div[1]/a"))).text
            Ratings_cnt = np.Nan
            driver.back()

        productdf.loc[product_index] = [Title, Price, Rating, Seller, Ratings_cnt]
        product_index += 1
        driver.close()
    #coming back to main page
    try:
        driver.switch_to.window(driver.window_handles[0])
        #checkking if next page exists
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="search"]/div[1]/div[1]/div/span[1]/div[1]/div[26]/div/div/span/a[1]'))).click()
        if len(productdf.index) >= 20:
            iterate = False
    except:
        break
        


In [5]:
t2 = time.time()
print(t2-t1)
display(productdf)

71.39015078544617


,Name,Price,Rating,Seller,Ratings_cnt
1001,Duracell 20000 MAH Slimmest Power Bank with 1 ...,2899,4.2,,"3,051 ratings"
1002,Portronics Luxcell MagClick 10k 10000 mAh 15W ...,1399,4.0,Cocoblu Retail,"3,777 ratings"
1003,Portronics Luxcell B 20K Advanced 20000 mAh Po...,1149,4.0,Cocoblu Retail,"3,777 ratings"
1004,Amazon Basics 20000 mAh Power Bank | 22.5W Fas...,999,4.0,Clicktech Retail Private Ltd,565 ratings
1005,boAt Newly Launched Pocket Energyshroom PB400 ...,1899,4.2,Cocoblu Retail,149 ratings
1006,"ZEBRONICS MW63 Wireless Power Bank, 10000 mAh,...",999,3.5,Cocoblu Retail,454 ratings
1007,Portronics Luxcell B 10K 10000 mAh 22.5W Fast ...,699,4.0,Cocoblu Retail,"1,123 ratings"
1008,Xiaomi Power Bank 4i 20000mAh 33W Super Fast C...,1999,4.3,Cocoblu Retail,"94,876 ratings"
1009,Xiaomi Power Bank 4i 20000mAh 33W Super Fast C...,1899,4.3,Cocoblu Retail,"94,876 ratings"
1010,Portronics Luxcell Wireless Mini 10k 10000mAh ...,1249,4.0,Cocoblu Retail,"3,777 ratings"


In [6]:
# transfering to csv file
productdf.to_csv('Data.csv')
driver.quit()